# Ejemplo Completo: Clasificación Supervisada
## Clasificación de Especies de Flores Iris

### Objetivo
Clasificar flores Iris en tres especies diferentes basándose en medidas de pétalos y sépalos.

### Conceptos que aprenderás:
- Trabajar con datasets reales incluidos en Scikit-Learn
- Implementar clasificación con Random Forest
- Evaluar clasificadores (accuracy, precision, recall, F1)
- Interpretar matrices de confusión
- Analizar importancia de características
- Calcular probabilidades de predicción

---

## 1. Importar Bibliotecas

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, 
    classification_report, 
    confusion_matrix,
    ConfusionMatrixDisplay,
    precision_score,
    recall_score,
    f1_score
)
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración para gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("Set2")

---

## 2. Cargar y Explorar el Dataset

El dataset Iris es uno de los más famosos en Machine Learning. Contiene medidas de 150 flores de 3 especies diferentes.

In [ ]:
# Cargar el dataset clásico de Iris
iris = load_iris()

# Estructura del dataset
print("=" * 70)
print("DATASET IRIS - INFORMACIÓN GENERAL")
print("=" * 70)
print(f"\nDescripción: {iris.DESCR[:200]}...")
print(f"\nNúmero de muestras: {iris.data.shape[0]}")
print(f"Número de características: {iris.data.shape[1]}")
print(f"Clases: {iris.target_names}")
print(f"Nombres de características: {iris.feature_names}")

In [ ]:
# Crear DataFrame para mejor visualización
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['especie'] = pd.Categorical.from_codes(iris.target, iris.target_names)

print("\n" + "=" * 70)
print("PRIMERAS 10 FILAS DEL DATASET")
print("=" * 70)
print(df.head(10))

print("\n" + "=" * 70)
print("ÚLTIMAS 10 FILAS DEL DATASET")
print("=" * 70)
print(df.tail(10))

# Guardar el dataset en un archivo CSV
# df.to_csv('iris_dataset.csv', index=False)
print("\nEl dataset se ha guardado en 'iris_dataset.csv'.")

---

## 3. Análisis Exploratorio de Datos (EDA)

In [ ]:
print("\n" + "=" * 70)
print("ESTADÍSTICAS DESCRIPTIVAS")
print("=" * 70)
print(df.describe())

print("\n" + "=" * 70)
print("DISTRIBUCIÓN DE CLASES")
print("=" * 70)
print(df['especie'].value_counts())
print(f"\nDataset balanceado: {df['especie'].value_counts().nunique() == 1}")

In [ ]:
# Visualización de la distribución de clases
plt.figure(figsize=(8, 5))
df['especie'].value_counts().plot(kind='bar', color=['skyblue', 'lightcoral', 'lightgreen'])
plt.title('Distribución de Especies de Iris', fontsize=14, fontweight='bold')
plt.xlabel('Especie')
plt.ylabel('Cantidad de Muestras')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Visualización de características por especie
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

caracteristicas = iris.feature_names
for idx, feature in enumerate(caracteristicas):
    ax = axes[idx // 2, idx % 2]
    for especie in iris.target_names:
        data = df[df['especie'] == especie][feature]
        ax.hist(data, alpha=0.6, label=especie, bins=15)
    ax.set_xlabel(feature)
    ax.set_ylabel('Frecuencia')
    ax.set_title(f'Distribución: {feature}')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Pairplot para ver relaciones entre características
print("\nGenerando gráfico de relaciones entre características...")
sns.pairplot(df, hue='especie', diag_kind='hist', markers=['o', 's', 'D'])
plt.suptitle('Relaciones entre Características por Especie', y=1.02, fontsize=14)
plt.show()

In [ ]:
# Matriz de correlación
print("\n" + "=" * 70)
print("MATRIZ DE CORRELACIÓN")
print("=" * 70)
correlation_matrix = df.drop('especie', axis=1).corr()
print(correlation_matrix)

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Matriz de Correlación entre Características')
plt.tight_layout()
plt.show()

---

## 4. Preparar los Datos

In [ ]:
# Separar características (X) y variable objetivo (y)
X = iris.data
y = iris.target

print("=" * 70)
print("PREPARACIÓN DE DATOS")
print("=" * 70)
print(f"\nForma de X (características): {X.shape}")
print(f"Forma de y (objetivo): {y.shape}")

# Dividir en conjunto de entrenamiento (70%) y prueba (30%)
# stratify=y asegura que ambos conjuntos tengan la misma proporción de clases
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y  # Importante para mantener balance
)

print(f"\nDatos de entrenamiento: {X_train.shape[0]} muestras")
print(f"Datos de prueba: {X_test.shape[0]} muestras")
print(f"Proporción: {X_train.shape[0]/len(X)*100:.1f}% train, {X_test.shape[0]/len(X)*100:.1f}% test")

# Verificar balance en ambos conjuntos
print("\nDistribución de clases en entrenamiento:")
unique, counts = np.unique(y_train, return_counts=True)
for clase, count in zip(iris.target_names[unique], counts):
    print(f"  {clase}: {count}")

print("\nDistribución de clases en prueba:")
unique, counts = np.unique(y_test, return_counts=True)
for clase, count in zip(iris.target_names[unique], counts):
    print(f"  {clase}: {count}")

---

## 5. Escalar las Características

Importante: Random Forest no requiere escalado, pero lo hacemos para demostrar buenas prácticas.

In [ ]:
# Escalar características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n" + "=" * 70)
print("ESCALADO DE CARACTERÍSTICAS")
print("=" * 70)
print("\nEstadísticas ANTES del escalado (conjunto de entrenamiento):")
print(f"Media: {X_train.mean(axis=0)}")
print(f"Desviación estándar: {X_train.std(axis=0)}")

print("\nEstadísticas DESPUÉS del escalado:")
print(f"Media: {X_train_scaled.mean(axis=0)}")
print(f"Desviación estándar: {X_train_scaled.std(axis=0)}")

---

## 6. Crear y Entrenar el Modelo

In [ ]:
# Crear el modelo de Random Forest
modelo_clasificador = RandomForestClassifier(
    n_estimators=100,      # Número de árboles
    max_depth=5,           # Profundidad máxima de cada árbol
    random_state=42,       # Reproducibilidad
    n_jobs=-1              # Usar todos los cores del CPU
)

print("\n" + "=" * 70)
print("ENTRENAMIENTO DEL MODELO")
print("=" * 70)
print("Configuración del modelo:")
print(f"  • Algoritmo: Random Forest Classifier")
print(f"  • Número de árboles: {modelo_clasificador.n_estimators}")
print(f"  • Profundidad máxima: {modelo_clasificador.max_depth}")
print("\nEntrenando modelo...")

# Entrenar el modelo
modelo_clasificador.fit(X_train_scaled, y_train)

print("✓ Modelo entrenado exitosamente!")

---

## 7. Hacer Predicciones

In [ ]:
# Hacer predicciones en el conjunto de prueba
y_pred = modelo_clasificador.predict(X_test_scaled)

# También podemos obtener probabilidades
y_pred_proba = modelo_clasificador.predict_proba(X_test_scaled)

# Imprimir predicciones con .predict()
print("\n" + "=" * 70)
print("PREDICCIONES con .predict()")
print("=" * 70)
print("\nPrimeras 5 predicciones:")
print("-" * 70)
for i in range(5):
    real = iris.target_names[y_test[i]]
    pred = iris.target_names[y_pred[i]]
    correcto = "✓" if real == pred else "✗"
    print(f"{correcto} Real: {real:15s} | Predicho: {pred:15s}")


# Imprimir predicciones con .predict_proba()
print("\n" + "=" * 70)
print("PREDICCIONES con .predict_proba()")
print("=" * 70)
print("\nPrimeras 5 predicciones con probabilidades:")
print("-" * 70)
for i in range(5):
    real = iris.target_names[y_test[i]]
    proba = y_pred_proba[i]
    proba_str = ", ".join([f"{iris.target_names[j]}: {p:.2f}" for j, p in enumerate(proba)])
    print(f"Real: {real:15s} | Probabilidades: {proba_str}")


---

## 8. Evaluar el Modelo

### Métricas de evaluación para clasificación

- **Accuracy**: Proporción de predicciones correctas sobre el total de muestras.
- **Precision**: Proporción de verdaderos positivos entre todos los positivos predichos.
- **Recall**: Proporción de verdaderos positivos entre todos los positivos reales.
- **F1-Score**: Promedio armónico entre precisión y recall, útil en clases desbalanceadas.

#### ¿Qué es el F1-Score?

El **F1-Score** es una métrica que combina precisión y recall en un solo valor. Es especialmente útil cuando las clases están desbalanceadas o cuando los falsos positivos y falsos negativos tienen distinto impacto.

Se calcula como el promedio armónico entre precisión y recall:

$$
F1 = 2 \cdot \frac{\text{Precisión} \cdot \text{Recall}}{\text{Precisión} + \text{Recall}}
$$

### Rango del F1-Score

El **F1-Score** varía entre **0 y 1**:

- **F1 = 1**: rendimiento perfecto (precisión y recall son ambos 1).
- **F1 = 0**: el modelo no detectó ningún positivo correctamente.

Este valor refleja el **balance entre precisión y recall**. Si uno de los dos es bajo, el F1 también será bajo, incluso si el otro es alto.



### ¿Por qué usar F1?

- Penaliza los desequilibrios entre precisión y recall.
- Es más conservador que el promedio aritmético.
- Ideal cuando se necesita un balance entre detectar correctamente y evitar errores.

### Ejemplo

Si un modelo tiene:
- Precisión = 0.9
- Recall = 0.6

Entonces:

$$
F1 = 2 \cdot \frac{0.9 \cdot 0.6}{0.9 + 0.6} = 0.72
$$

Aunque la precisión es alta, el F1 baja porque el recall es bajo.


In [ ]:
# Calcular métricas principales
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("\n" + "=" * 70)
print("RESULTADOS DEL MODELO")
print("=" * 70)
print(f"\nExactitud (Accuracy): {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"Precisión (Precision): {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

print("\n" + "-" * 70)
print("INTERPRETACIÓN:")
print("-" * 70)
print(f"• De cada 100 predicciones, {accuracy*100:.0f} son correctas")
if accuracy > 0.95:
    print("  → Excelente rendimiento del modelo")
elif accuracy > 0.85:
    print("  → Muy buen rendimiento del modelo")
elif accuracy > 0.75:
    print("  → Buen rendimiento del modelo")
else:
    print("  → El modelo necesita mejoras")

In [ ]:
# Reporte de clasificación detallado
print("\n" + "=" * 70)
print("REPORTE DE CLASIFICACIÓN DETALLADO")
print("=" * 70)
print(classification_report(y_test, y_pred, target_names=iris.target_names))

print("\n" + "-" * 70)
print("EXPLICACIÓN DE MÉTRICAS:")
print("-" * 70)
print("• Precision: De las predicciones de una clase, cuántas son correctas")
print("• Recall: De todos los casos reales de una clase, cuántos detectamos")
print("• F1-Score: Media armónica entre precision y recall")
print("• Support: Número de muestras reales de cada clase")

---

## 9. Matriz de Confusión

In [ ]:
# Calcular matriz de confusión
cm = confusion_matrix(y_test, y_pred)

print("\n" + "=" * 70)
print("MATRIZ DE CONFUSIÓN")
print("=" * 70)
print("\nMatriz (valores absolutos):")
print(cm)

# Visualizar matriz de confusión
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Matriz con valores absolutos
disp1 = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=iris.target_names)
disp1.plot(ax=axes[0], cmap='Blues', values_format='d')
axes[0].set_title('Matriz de Confusión (Valores Absolutos)', fontweight='bold')

# Matriz normalizada (porcentajes)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
disp2 = ConfusionMatrixDisplay(confusion_matrix=cm_normalized, display_labels=iris.target_names)
disp2.plot(ax=axes[1], cmap='Blues', values_format='.2%')
axes[1].set_title('Matriz de Confusión (Normalizada)', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n" + "-" * 70)
print("INTERPRETACIÓN DE LA MATRIZ:")
print("-" * 70)
print("• Diagonal principal: Predicciones correctas")
print("• Fuera de la diagonal: Errores de clasificación")
print("• Cada fila suma el total de casos reales de esa clase")

---

## 10. Importancia de Características

In [ ]:
# Obtener importancia de características
# devuelve un array con valores entre 0 y 1, que suman 1.
importancias = modelo_clasificador.feature_importances_

# Crear DataFrame para mejor visualización
df_importancias = pd.DataFrame({
    'caracteristica': iris.feature_names,
    'importancia': importancias
}).sort_values('importancia', ascending=False)

print("\n" + "=" * 70)
print("IMPORTANCIA DE CARACTERÍSTICAS")
print("=" * 70)
print(df_importancias)
print("\nCaracterística más importante:", df_importancias.iloc[0]['caracteristica'])

In [ ]:
# Visualizar importancia de características
plt.figure(figsize=(10, 6))
plt.barh(df_importancias['caracteristica'], df_importancias['importancia'], color='skyblue')
plt.xlabel('Importancia')
plt.title('Importancia de Características en el Modelo', fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

---

## 11. Predicción con Nuevas Flores

In [ ]:
print("\n" + "=" * 70)
print("PREDICCIÓN PARA NUEVAS FLORES")
print("=" * 70)

# Crear nuevas flores para clasificar
nuevas_flores = np.array([
    [5.1, 3.5, 1.4, 0.2],  # Características similares a setosa
    [6.5, 3.0, 5.5, 1.8],  # Características similares a virginica
    [5.7, 2.8, 4.1, 1.3]   # Características similares a versicolor
])

# Escalar las nuevas flores
nuevas_flores_scaled = scaler.transform(nuevas_flores)

# Hacer predicciones
predicciones = modelo_clasificador.predict(nuevas_flores_scaled)
probabilidades = modelo_clasificador.predict_proba(nuevas_flores_scaled)

# Mostrar resultados
for i, (flor,pred, probs) in enumerate(zip(nuevas_flores, predicciones, probabilidades), 1):
    print(f"\n{'=' * 70}")
    print(f"FLOR {i}")
    print('=' * 70)
    print(f"Características:")
    for nombre, valor in zip(iris.feature_names, flor):
        print(f"  • {nombre}: {valor}")
        print(f"\n→ Especie predicha: {iris.target_names[pred]}")
        print(f"\nProbabilidades por clase:")
    for nombre, prob in zip(iris.target_names, probs):
        barra = '█' * int(prob * 50)
        print(f"  {nombre:15s}: {prob:.4f} ({prob*100:5.2f}%) {barra}")

---

## 12. Análisis de Errores

In [ ]:
# Identificar casos mal clasificados
errores = y_test != y_pred
indices_errores = np.where(errores)[0]

print("\n" + "=" * 70)
print("ANÁLISIS DE ERRORES")
print("=" * 70)
print(f"\nTotal de errores: {errores.sum()} de {len(y_test)} ({errores.sum()/len(y_test)*100:.2f}%)")

if errores.sum() > 0:
    print("\nCasos mal clasificados:")
    print("-" * 70)
    for idx in indices_errores:
        real = iris.target_names[y_test[idx]]
        pred = iris.target_names[y_pred[idx]]
        print(f"\nÍndice {idx}:")
        print(f"  Real: {real}")
        print(f"  Predicho: {pred}")
        print(f"  Características: {X_test[idx]}")
        print(f"  Probabilidades: {y_pred_proba[idx]}")
else:
    print("\n¡Perfecto! El modelo clasificó correctamente todas las muestras.")

---

## 13. Validación Cruzada

In [ ]:
from sklearn.model_selection import cross_val_score, cross_validate

print("\n" + "=" * 70)
print("VALIDACIÓN CRUZADA (5 FOLDS)")
print("=" * 70)

# Realizar validación cruzada
cv_scores = cross_val_score(
    modelo_clasificador, 
    X_train_scaled, 
    y_train, 
    cv=5,
    scoring='accuracy'
)

print(f"\nScores por fold:")
for i, score in enumerate(cv_scores, 1):
    print(f"  Fold {i}: {score:.4f} ({score*100:.2f}%)")

print(f"\n{'=' * 70}")
print(f"Accuracy promedio: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")
print(f"Rango: [{cv_scores.min():.4f}, {cv_scores.max():.4f}]")

In [ ]:
# Validación cruzada con múltiples métricas
print("\n" + "=" * 70)
print("VALIDACIÓN CRUZADA CON MÚLTIPLES MÉTRICAS")
print("=" * 70)

scoring = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']
cv_results = cross_validate(
    modelo_clasificador,
    X_train_scaled,
    y_train,
    cv=5,
    scoring=scoring
)

for metrica in scoring:
    scores = cv_results[f'test_{metrica}']
    print(f"\n{metrica.replace('_', ' ').title()}:")
    print(f"  Media: {scores.mean():.4f} ± {scores.std():.4f}")

---

## 14. Comparación con Otros Modelos

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

print("\n" + "=" * 70)
print("COMPARACIÓN CON OTROS ALGORITMOS")
print("=" * 70)

# Definir varios modelos
modelos = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'SVM': SVC(random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=200)
}

In [ ]:
# Entrenar y evaluar cada modelo
resultados = []

for nombre, modelo in modelos.items():
    # Entrenar
    modelo.fit(X_train_scaled, y_train)
    
    # Predecir
    y_pred_modelo = modelo.predict(X_test_scaled)
    
    # Evaluar
    acc = accuracy_score(y_test, y_pred_modelo)
    prec = precision_score(y_test, y_pred_modelo, average='weighted')
    rec = recall_score(y_test, y_pred_modelo, average='weighted')
    f1 = f1_score(y_test, y_pred_modelo, average='weighted')
    
    resultados.append({
        'Modelo': nombre,
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1-Score': f1
    })

# Crear DataFrame con resultados
df_resultados = pd.DataFrame(resultados).sort_values('Accuracy', ascending=False)
print("\n", df_resultados.to_string(index=False))

In [ ]:
# Visualizar comparación
plt.figure(figsize=(12, 6))
x = np.arange(len(df_resultados))
width = 0.2

plt.bar(x - 1.5*width, df_resultados['Accuracy'], width, label='Accuracy', alpha=0.8)
plt.bar(x - 0.5*width, df_resultados['Precision'], width, label='Precision', alpha=0.8)
plt.bar(x + 0.5*width, df_resultados['Recall'], width, label='Recall', alpha=0.8)
plt.bar(x + 1.5*width, df_resultados['F1-Score'], width, label='F1-Score', alpha=0.8)

plt.xlabel('Modelos')
plt.ylabel('Score')
plt.title('Comparación de Modelos de Clasificación', fontweight='bold')
plt.xticks(x, df_resultados['Modelo'], rotation=45, ha='right')
plt.ylim([0.8, 1.0])
plt.legend()
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Elegir el mejor modelo basado en F1-Score
mejor_modelo = max(resultados, key=lambda x: x['F1-Score'])
print("\n" + "=" * 70)
print("MEJOR MODELO SEGÚN F1-SCORE")
print("=" * 70)
print(f"\nModelo: {mejor_modelo['Modelo']}")
print(f"Accuracy: {mejor_modelo['Accuracy']:.4f}")
print(f"Precision: {mejor_modelo['Precision']:.4f}")
print(f"Recall: {mejor_modelo['Recall']:.4f}")
print(f"F1-Score: {mejor_modelo['F1-Score']:.4f}")


## 15. Conclusiones

### ✓ Lo que aprendimos:
1. **Cargar datasets reales** incluidos en Scikit-Learn
2. **Implementar clasificación** con Random Forest
3. **Evaluar clasificadores** usando múltiples métricas
4. **Interpretar matrices de confusión** para entender errores
5. **Analizar importancia de características**
6. **Calcular probabilidades** de predicción
7. **Validación cruzada** para evaluación robusta
8. **Comparar diferentes algoritmos**

### 📊 Resultados clave:
- El modelo Random Forest logra excelente accuracy en este dataset
- Las características de pétalos son más importantes que las de sépalos
- La matriz de confusión muestra dónde ocurren los errores
- La validación cruzada confirma la robustez del modelo

### 🎯 Aplicaciones reales:
Este tipo de clasificación se usa en:
- **Medicina**: Diagnóstico de enfermedades
- **Finanzas**: Clasificación de riesgo crediticio
- **Marketing**: Segmentación de clientes
- **Manufactura**: Control de calidad
- **Biología**: Clasificación de especies

### 💡 Próximos pasos:
- Probar con datasets desbalanceados
- Explorar técnicas de feature engineering
- Aplicar ensemble methods más avanzados
- Optimizar hiperparámetros con GridSearchCV
- Trabajar con datos del mundo real (Kaggle)

---

## 📚 Referencias
- Dataset Iris: Fisher, R.A. (1936). "The use of multiple measurements in taxonomic problems"
- Documentación Scikit-Learn: https://scikit-learn.org/stable/modules/ensemble.html
- Random Forest: Breiman, L. (2001). "Random Forests"
